# Data intelligence application
## Pricing & Advertising project

## 0. Introduction
This report aims to give an overview of the steps conducted to reach the goal of modelling the following given scenario: consider the scenario in which advertisement is used to attract users on an ecommerce website and the users, after the purchase of the first unit of a consumable item, will buy additional units of the same item in future. We will walk through the description of the specific scenario we have studied and the design choices we adopted in order to yield the optimal solution for our obejctive: to find the best joint bidding and pricing strategy.


## 1.Scenario
The real world scenario we chose to model is the exploitation of online advertising to enhance the incomes of a **coffee** seller and the consequent choice of the best price for each class of costumer. For the sake of simiplicity the costumers of the coffee website are supposed to be categorized based on a feature space described by to 2 binary features:
* Customers used to buy online
* Every day consumers
|       | Every day | Not Every day |
|-------|----------|--------------|
| **Online buyer** | C1        | C3            |
| **Not Online buyer** | C3        | C2             |


The table above shows the 3 sub-categories that distinguish the customers:
* **C1**: customers used to buy online and that consume coffee on a daily basis
* **C2**: class that includes two kinds of customers since they behave in a similar manner, that is the every day consumers that are not very accostumed to buy online and the customers that drink coffee less frequently but that buy often online
* **C3**: hard customers that are not used to buy online and that drink coffee less frequently than every day consumers

<br>

Each customers' class is characterized by:
* a stochastic **number of daily clicks** of new users as a function depending on the bid;
* a stochastic **cost per click** as a function of the bid;
* a **conversion rate** function providing the probability that a user will buy the item given a price;
* a distribution probability over the **number of times the user will come back** to the ecommerce website to buy that item by **30 days** after the first purchase.

For the aforementioned class' characteristics we have retrieved some data on similar items from the internet and with some work of averaging and adjustment the following distributions have been generated:<br><br>
**INSERT GRAPHS**<br><br>

The data are generated by the class _StandardGenerator_ that handles the reading process of the parameters and values from a file in _.json_ format.<br>
In particular the number of daily clicks, the cost per click and the future purchases of the user are modelled with the following functions:

**sta tabella del cazzo non funge**

| Variable | Function |
|---------:|:---------|
| daily clicks | $U_B * (1 - e^{(-f_S * x)})$ |
| cost per click | $C * log(1 + x/C)$ | 
| future purchases | $?$ |


## 2. Model formalization
Once we have described the scenario in which we are going to work, we need to formalize in a mathematical fashion which is our goal, how we want to achieve it and the variables that come into play.<br> 
### 2a. Assumptions
1. Every price available is associated with a margin obtained by the sale that is known beforehand. Without loss of generality we assume to have a fixed cost for the coffee production and so to use directly the margin for the pricing process<br>
2. The price _p_ for a given costumers is fixed for any future purchases<br>
3. If a customer visit again the website after a first purchase, he/she will surely buy the item again<br>
4. Ther is no budget constraint<br>
5. Auctions are not included in the modelling of the scenario nor simulated<br>
6. Every sub-campaign is asssociated with a given class of customer (i.e. context)<br>
7. The considered time horizon is of 1 year

### 2b. Offline setting
In this section the problem is formulated by mean of an objective function followed by the formalization of an algortihm to solve it. Here we make a further **assumption** of working in an **offline setting**: the values of the parameters of the problem are known. Before stating the function, the parameters with their descriptions and the notations are given in the table below.

| Variable | Description |
|---------:|:------------|
| $t$ | Time/Day  |
| $T$ | Time horizon |
| $j$ | A customer class |
| $N$ | Number of customer classes |
| $p_{j,t}$ | Price at time $t$ for the customer class $j$ |
| $c_{j}$ | Conversion rate at price $p$ for the customer class $j$ |
| $m$ | Margin obtained by the sale at price $p$ |
| $x_{j,t}$ | Bid of subcampaign $j$ at time $t$ |
| $n_{j}$ | Number of clicks of new users of subcampaign $j$, given the value of the bid $x_{j,t}$ |
| $\tau_{j}$ | Number of times the user buy again the item by 30 days after the first purchase |
| $CPC_{j}$| Cost per click for the subcampaign $j$, given the value of the bid $x_{j,t}$ |

<br><br>
Now we can formulate our objective function for finding the best **joint pricing and bidding strategy** with the goal of **maximizing the profit**:
<br><br>
> $ \underset{p_{j,t} , x_{j,t}} {\textrm{max}} \sum \limits _{t} ^{T} \sum \limits _{j} ^{N} n_{j}(x_{j,t}) \text{  } [ \text{  } c_{j}(p_{j,t}) \text{  } m(p_{j,t}) \text{  } (\tau_{j} + 1) - CPC_{j}(x_{j,t}) ] $
<br><br>

The algorithm to solve the problem could be a classiacl brute-force approach based on the following setting:<br><br>
For $t = 1$ to $T$, and for $j = 1$ to $N$ set:
> $p_{j,t}^{*}, x_{j,t}^{*} = \underset{p_{j,t} , x_{j,t}}{\textrm{arg max}} \text{  } n_{j}(x_{j,t}) \text{  } [ \text{  } c_{j}(p_{j,t}) \text{  } m(p_{j,t}) \text{  } \tau_{j} - CPC_{j}(x_{j,t}) ]$

The values of all the parameters are known and available in $\Theta(1)$, given the assummption of an offline setting.  
Defining $X$ the total number of bids and $P$ the total number of prices, the algorithm finds the optimal values $p_{j,t}^{*}, x_{j,t}^{*}$ with time complexity: $$ \Theta(T \text{ } N \text{ } X \text{ } P) $$


### 2c. Online setting
Once the offline version of the model has been formalized and the scenario depicted, the scope of the project resides the most in solving a real case scenario that is best modelled by an **online setting**. Here we remove the previous assumption of knowing beforhand all the parameters of the distributions. This means that we have to find the best values for the price and bid, **without having full knowledge of the distributions** governing our variables. To find an approximation of the variables we have to sample values from the environment and build an increasingly-better estimation of the underlying distribution.
Each day is considered a different "round" of our problem and, as previously stated, we consider a time horizon of 1 year.<br>

##### Random Variables
In this section we list the variables of which we do not have full knowledge. These variables will be sampled at each round from a distribution in the environment. 

| Random Variable | Motivation |
|---------:|:------------|
| $CPC_{j}$| Cost per click is randomly extracted from distribution |
| $c_{j}$ | The conversion (buying an item after visiting the site) is sampled from a distribution |
| $n_{j}$| Number of new users is randomly extracted at the 'start' of each day |
| $\tau_{j}$| The number of times the user buys again is sampled from a distr. after the first purchase |

Sampling from a variable means extracting a value from a distribution with the chosen parameter.
In particular when sampling from the Conversion Rate, we sample from a _Bernoulli_ distribution. Meanwhile, when sampling from the CPC and the number of new users we sample from a _Gaussian_ distribution with the chosen mean and variance parameters. Finally, the number of future purchases are sampled from a _Binomial_ distribution.

##### Delays
The anlysis of the problem lead also to consider potential delays in the feedbacks. <br>
The delay that can be considered in the given scenario is that the subsequent buys from the same user are not considered instantaneous but delayed by a certain time. For the sake of simplicity, we considered the time to be fixed to 30 days, but it can also be implemented as a random variable where the delay sampled from a probability distribution. 

| Delay | Description |
|---------:|:------------|
| $\alpha_{j}$| Delay in acquiring item again  | 

To cope with the delay issue we have chosen to draw a sample from a Gaussian distribution each time a user buy the coffee on the website. This sample represents the number of time the user will come back to buy the item again. After 30 rounds from the purchase, the income of the future purchases is added to the reward linked to the strategy chosen by the algorithm in order to update the parameters.

### 2d. Algorithm
To solve the online optimization problem we use a MAB approach.
In the MAB approach the objective is to minimize the regret, defined as the cumulative difference between the reward of the clairvoyant algorithm, which always chooses the optimal arm $\mu^*$, and the reward given by the arm which we choose at a specific round $\mu$.<br> 
Each arm is described by a given value that depends on the problem we are trying to solve: 
- a bid value in case of advertising problem
- a price value for a pricing problem

The mathematical formualation of the MAB setting for the scenario is the following:
> $ \underset{p_{j,t} , x_{j,t}} \min \rho $
<br>
<br>
> $ \rho = T \cdot \mu^* - \sum_{t=0}^T \mu_t $
<br>
<br>

The rewards use the values sampled from the distributions described in the previous section and follow the formula:
<br>
<br>
> $ \mu^* = {n_{j}^*} (x_{j,t}) \text{  } [ \text{  } c_{j}^*(p_{j,t}) \text{  } m(p_{j,t}) \text{  } (\tau_{j}^* + 1) - CPC^*_{j}(x_{j,t}) ] $
<br>
<br>
$ \mu_t = {n_{j,t}} (x_{j,t}) \text{  } [ \text{  } c_{j,t}(p_{j,t}) \text{  } m(p_{j,t}) \text{  } (\tau_{j,t} + 1) - CPC_{j,t}(x_{j,t}) ] $

##### Learner
The _Learner_ class is the abstract class from which the other learner algortithms, used in the different steps of the problem, inherit the structure. The key parts of the _Learner_' stucture goes from the initialization to the update of the parameters. Once a new _Learner_ is created, variables like the number of the arms and the value related to each arm are set to the values related to the problem we are going to solve (i.e. different price values in a pricing scenario); other variables like the time _t_ and the collected _rewards_ are initialized to 0 and to an empty list respectively. The basic operations of the _Learner_ class reside in the _pull_arm_ method that return the best sampled arm at the specific round and the _update_ method that handle the parameters to be modified after the round has been played.

##### Environment
The _CompleteEnvironment_ class aims to simulate a real environment. Given the distributions for the sub-class parameters it returns a sample of the values needed by the _Learner_ to update the collected rewards and the related parameters. The returned values are completely transparent to the _Learner_ that can retrieve the different values of $CPC_{j}$, $c_{j}$, $n_{j}$, $\tau_{j}$. This values are sampled from the distributions retrieved thanks to the _Standard__Generator_ class that reads them from the _json_ file.<br>

### 2e. Approach
The approach followed to solve our goal of finding the best joint pricing/bidding startegy is a step by step approach divided into **pricing campaign** and **advertising campaign** separated at first. Both campaigns start from the trivial scenario in which some paramaters are known beforhand and **fixed** and the optimization problem is solved by considering the **aggregate** classes. We then continue by integrting the **context** structure that handles the optimization problem for each subclass and finally we intergrate the different campaigns to solve the optimization problem in its entirety.
Further details on the learners, environment and other key classes to solve the problem will be given in the following sections.

## 5. Advertising campaing
Advertising is a marketing comunication that aim to promote to sell a product. In our specific scenario, advertising campaign is carried out by means of media and web channels. Given our assuptions, we are endowed of an unlimited budget and we do not need to model and deal with the auction mechanism that regulate the advertiser and auctioner correlation. For this reason we will focus only on the choice of the best bid to maximize the profit and optimize the objective function. 


### 5a. Scenario
In the advertising scenario we start by studying the **aggregate** case, that is, we do not distinguish between different classes of users. Moreover, since our optimization function includes some variables that depends on the price, we will assume it to be fixed to a given value chosen a priori. This results in having a fixed value also for the variables or the parameters of the distributions that depend on the price such as the conversion rate, the margin and the future purchases. To sum up, the goal of the advertising problem in the initial setting is to **maximize the profit with respect to the cost per click $CPC$ and the number of clicks $n$** depending on the chosen bid.

### 5b. Environment
The structure is more or less the one described in the previous sections. The only difference pertains the returned values as _reward_ to the learner of the advertising problem. The round played by the environment resides mostly in the sampling of the $CPC$ and the number of new users that are lured to click on the ads. The variables depending on the price are, as stated before, fixed.

### 5c. Learner
In this scenario the learners are modelled in order to try out different bid values in a MAB setting. Each arm of the learners is assigned to a different bid value. In our specific problem there will be 10 different bids; discretized to overcome the fact that it should be a continuos variable. For our problem we have chosen to initially use two different learner models: _Gaussian Thompson Sampling_ (GTS) and _Gaussian Process Thompson Sampling_ (GPTS).

##### GTS
For a detailed description of the _Thompson Sampling Algorithm_ see the _Learner_ section in the _Pricing campaign_ part. The only difference with respet to a classical TS approach is the use as a prior and as arms' distribution a _Gaussian_ distribution.

##### GPTS
The _Gaussian Process Thompson Sampling_ is based, as the name suggests, in the use of Gaussian Processes. Gaussian Processes are a generic supervised learning method designed to solve regression and probabilistic classification problems. GP models a set of random variables such that they all have a multivariate normal distribution. In particular our _GPTS Learner_ exploit a Gaussian Process regressor since it is trying to model the income based on the rewards yielded by the environment that depends on the arm chosen by the GPTS. A Gaussian Process is completely defined by its mean and its covariance. Since we do not have any prior information we assumed to have _zero_ mean and the covariance given by the squared exponential kernel function $k(x,x')$:
> $k(x,x') = \theta^2 e^{-\frac{(x-x')^2}{2l^2}}$

where:
* $l$ is the _lenghtscale_
* $\theta$ is the _scale factor_

The optimal value of the 2 hyperparameters have been found by maximization of the marginal likelihood of the fit process. Meanwhile the ranges for the hyperparameters has been set on an exeprimental basis. The fit process is performed each time a new observations is retrieved from the environment. In this way the GPTS manages to reduce the uncertainty of its estimations.

##### Penalty
Both the learners are equipped with a mechanism that penalizes the arms that are too "risky". The learners avoid the arms that return an expected neagative value beyond a certain probability that we have fixed to 20%. To prevent the learners to get stucked in the initial phase (since each arm overcomes the threshold given the initial values), the algorithm starts by performing an explration phase of the arms available in order to have some information on the outcome of the pulling process.

### 5d. Performance